In [2]:
########################################
''' Imports '''

from sklearn.datasets import load_files       
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np
from glob import glob
import os, csv
import random
from random import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image 
from IPython.display import display 
from PIL import Image
from PIL import ImageFile 

from tqdm import tqdm

from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.optimizers import SGD
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.engine.topology import get_source_inputs

print("Import finished")

Import finished


In [3]:
########################################
''' Class '''

class Traffic_Sign:
    """ A Single Traffic sign model """
    
    def __init__(self, filePath, width, height, x1, y1, x2, y2, classID):
        self.filePath = filePath
        self.width = width
        self.height = height
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.classID = classID  
        

In [5]:
########################################
''' Print helper function '''

# pretty print Traffic Sign object
def ts_pretty_print(ts):
    print("File: " + ts.filePath + " has classID: " + ts.classID)
    


In [11]:
'''Mac'''
'''
TSName_txt_path = "/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/Dataset/reducedSetTS.txt"
label_map_output_path = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/obj_detection/data/TS_label_map.pbtxt"

# All standard pictures in this folder are with its class name and .png format e.g. A1A.png
standard_folder = "/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/Standards"
standard_img_extension = ".png"
'''

'''Windows'''
TSName_txt_path = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset/reducedSetTS.txt"
label_map_output_path = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/obj_detection/data/TS_label_map.pbtxt"

# All standard pictures in this folder are with its class name and .png format e.g. A1A.png
standard_folder = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Standards/"
standard_img_extension = ".png"

# dataset path
dataset_directory = "C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset"
training_dataset_directory = os.path.join(dataset_directory, "Training")
testing_dataset_directory = os.path.join(dataset_directory, "Testing")

In [9]:
########################################
''' Load dataset into variable '''
    

# function to load training, test, and validation datasets into variables 
def load_dataset(path):
    all_files = []
    for eachfolder in os.listdir(path):
        abs_loc = path + "/" + eachfolder
        if not os.path.isdir(abs_loc):
            continue

        #open csv for this folder
        with open(abs_loc + "/GT-" + eachfolder + ".csv", "r") as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=';')
            next(csv_reader, None) # skip header
            
            #create traffic sign object
            for row in csv_reader:
                abs_file_loc = abs_loc + "/" + row[0]
                all_files.append(Traffic_Sign(abs_file_loc, row[1], row[2], row[3], row[4], row[5], row[6], row[7]))

            csv_file.close()
    return all_files




# load train and test datasets into list of objects of Traffic signs
#all_training_objs = load_dataset("/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/Dataset/Training") # a list of training traffic_sign_collection
#all_testing_and_validation_objs = load_dataset("/Users/kevinsu/Desktop/Udacity/machine-learning-master/projects/capstone/TSR/Dataset/Testing")  # a list of testing traffic_sign_collection

all_training_objs = load_dataset(training_dataset_directory) # a list of training traffic_sign_collection
all_testing_and_validation_objs = load_dataset(testing_dataset_directory)  # a list of testing traffic_sign_collection

# shuffle training objects and testing objects 
shuffle(all_training_objs)
shuffle(all_testing_and_validation_objs)



In [12]:
''' TS_label_map generator '''

def label_generator(TSName_62_collection):
    label_map_content = ""
    num_of_classes = 62
    for i in range(num_of_classes):
        label_map_content += "item{\n"
        label_map_content += "  id: {}\n".format(str(i+1))
        label_map_content += "  name: {}\n".format(TSName_62_collection[i])
        label_map_content += "}\n\n"
        
    with open(label_map_output_path, "w") as fout:
        fout.write(label_map_content) 

        
# create classID and className pairs for 62 Traffic Signs
with open(TSName_txt_path) as f:
    TSName_62 = f.readlines()
# remove whitespace characters
TSName_62 = [x.strip() for x in TSName_62[0:]]


# Generate label map for obj_detection
label_generator(TSName_62)


In [14]:
''' transfer ppm to png according to Tensorflow API '''

#coding=utf-8 
from PIL import Image 
import os


def convert_ppm_to_png(png_dataset_dir, ppm_dataset_directory):
    # if this path doesn't exist, create 
    if not os.path.exists(png_dataset_dir):
        os.makedirs(png_dataset_dir)
        
    for each_folder in os.listdir(ppm_dataset_directory):#e.g. 00000/
        each_folder = os.path.join(ppm_dataset_directory, each_folder)
        if not os.path.isdir(each_folder):
            continue
        for each_ppm in os.listdir(each_folder):#e.g. 00000/00001_00000.ppm
            if each_ppm[-3:] == "ppm":
                img_path = os.path.join(each_folder, each_ppm)
                img = Image.open(img_path)
                png_path = os.path.join(png_dataset_dir, each_ppm[:-3]+'png')
                print(png_path)
                img.save(png_path)
                #img.show()
        print(each_folder + " has all converted to png")
        
# Store all images in png under training/testing folder
png_training_dataset_dir = os.path.join(dataset_directory, 'png_training')
png_testing_dataset_dir = os.path.join(dataset_directory, 'png_testing')
        
convert_ppm_to_png(png_training_dataset_dir, training_dataset_directory)
convert_ppm_to_png(png_testing_dataset_dir, testing_dataset_directory)

C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01153_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01153_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01153_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01160_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01160_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01160_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01797_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01797_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01797_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recogniti

C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00965_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00965_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00965_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01028_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01028_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01028_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01103_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01103_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01103_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recogniti

C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\Training\00003 has all converted to png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00145_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00145_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00145_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00214_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00214_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00214_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00282_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00282_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sig

C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00618_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00618_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00689_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00689_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00689_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00689_00003.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00719_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00719_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00719_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recogniti

C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01397_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01397_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01397_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01406_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01406_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01406_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01423_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01423_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01426_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recogniti

...

C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00044_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00045_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00045_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00045_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00046_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00046_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00050_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00050_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00050_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recogniti

C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00952_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00957_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00957_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00957_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00959_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00959_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\00959_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01002_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01002_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recogniti

C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01275_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01275_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01275_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01276_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01276_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01276_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01502_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01502_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01502_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recogniti

C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01894_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01909_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01909_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01909_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01910_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01910_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01910_00002.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01911_00000.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\png_training\01911_00001.png
C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recogniti

NotADirectoryError: [WinError 267] The directory name is invalid: 'C:/Users/ksu/Desktop/ML/TSR_capstone/MLND_Traffic_Sign_Recognition/Dataset\\Training\\Readme.txt'

In [ ]:
########################################
''' Load the definition (standards) of our 62 traffic sign images '''
''' from 210 total traffic sign image dataset '''
def show_my_image(img_path):
    img=mpimg.imread(img_path)
    imgplot = plt.imshow(img)
    plt.show()
    
    


# display some standard images
for i in range(5):
    img_name = TSName_62[random.randint(0, 61)] + standard_img_extension
    img_path = os.path.join(standard_folder, img_name)
    show_my_image(img_path)
    print(img_path)
